In [1]:
import cv2,os
import numpy as np
from tqdm import tqdm

In [2]:
path = 'dataset/'
data = []
labels = []
bins=8
for folders in os.listdir(path):
    for folder in tqdm(os.listdir(path + folders)):
        items = path + folders + '/' + folder
        image = cv2.imread(items)
        image = cv2.resize(image,(64,64))
        image = image.flatten()
        data.append(image)
        if folders == 'cats':
            labels.append(0)
        else:
            labels.append(1)

100%|███████████████████████████████████████████████████████████████████████████| 12500/12500 [00:18<00:00, 668.34it/s]


In [3]:
import tensorflow.keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D , MaxPool2D , Flatten , Dropout 
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam

In [9]:
model = Sequential()

model.add(Dense(1024,input_shape=(64*64*3,)))
model.add(Dense(512,activation="relu"))
model.add(Dense(256,activation="relu"))
model.add(Dense(1, activation="sigmoid"))#sigmoid

model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_5 (Dense)              (None, 1024)              12583936  
_________________________________________________________________
dense_6 (Dense)              (None, 512)               524800    
_________________________________________________________________
dense_7 (Dense)              (None, 256)               131328    
_________________________________________________________________
dense_8 (Dense)              (None, 1)                 257       
_________________________________________________________________
Total params: 13,240,321
Trainable params: 13,240,321
Non-trainable params: 0
_________________________________________________________________


In [10]:
opt = Adam(lr=0.001)
model.compile(optimizer = opt , loss = 'binary_crossentropy' , metrics = ['accuracy'])

C:\Users\prave\AppData\Roaming\Python\Python38\site-packages\keras\optimizer_v2\adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [11]:
from sklearn.model_selection import train_test_split

data = np.array(data)
labels = np.array(labels)
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.25, random_state=42)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(18750, 12288) (6250, 12288) (18750,) (6250,)


In [12]:
history = model.fit(X_train,y_train, batch_size=256, epochs = 10, validation_data = (X_test, y_test))

Epoch 1/10
74/74 [==============================] - 2s 11ms/step - loss: 917.2526 - accuracy: 0.5061 - val_loss: 197.7889 - val_accuracy: 0.4974
Epoch 2/10
74/74 [==============================] - 1s 9ms/step - loss: 231.3267 - accuracy: 0.5018 - val_loss: 60.5616 - val_accuracy: 0.5110
Epoch 3/10
74/74 [==============================] - 1s 7ms/step - loss: 42.7375 - accuracy: 0.4950 - val_loss: 35.6033 - val_accuracy: 0.5026
Epoch 4/10
74/74 [==============================] - 1s 8ms/step - loss: 12.2368 - accuracy: 0.5091 - val_loss: 1.4640 - val_accuracy: 0.5530
Epoch 5/10
74/74 [==============================] - 1s 8ms/step - loss: 1.0752 - accuracy: 0.5372 - val_loss: 0.7312 - val_accuracy: 0.5469
Epoch 6/10
74/74 [==============================] - 1s 8ms/step - loss: 0.8106 - accuracy: 0.5481 - val_loss: 0.7490 - val_accuracy: 0.5368
Epoch 7/10
74/74 [==============================] - 1s 8ms/step - loss: 0.7273 - accuracy: 0.5433 - val_loss: 0.7197 - val_accuracy: 0.5325
Epoch 8/1

In [13]:
model.save('classification.h5')

In [14]:
from keras.models import load_model

model = load_model('classification.h5')

In [16]:
image = cv2.imread('dataset/cats/cat.6.jpg')
image = cv2.resize(image,(64,64))
image = image.flatten()
print(image.shape)
image = np.expand_dims(image,axis=0)
print(image.shape)
preds = model.predict(image)[0]
print(preds)

(12288,)
(1, 12288)
[0.49265343]
